In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import Ridge

from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.append('../../modules')
from helper_functions import feature_reduction_x, rating_feature_conversion


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jbyrneirl/BER-Rating-AI-Model/main/data/training/BERRatingData_aa.csv', sep=';', on_bad_lines="skip", low_memory=False)

np.random.seed(0)

#X = df.drop("BerRating", axis='columns')
#X = pd.get_dummies(X)
X = feature_reduction_x(df)
#print(len(list(X.columns)))

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)
y = df.BerRating


X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=2)

c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Aidan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [3]:
model = Ridge()


model.fit(X_train , y_train)
score = model.score(X_test , y_test)

y_hat = model.predict(X_test)

y_con = rating_feature_conversion(y_test.to_numpy())

In [4]:
print("Score: ", score, "\n")

y_hat_con = rating_feature_conversion(y_hat)
cm = confusion_matrix(y_con, y_hat_con)

print("CM")
print(cm)

print(accuracy_score(y_con, y_hat_con))

Score:  0.8624918296739676 

CM
[[  1   6   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [ 38 163  58   5   5   0   0   0   0   0   0   0   0   0   0]
 [ 28  99 171  72  18   2   0   0   0   0   0   0   0   0   0]
 [  2   8  10  27  18   4   2   0   0   0   0   0   0   0   0]
 [  1   2   6  25  47  25   8   7   3   1   0   0   0   1   0]
 [  1   2   2   3  33 102  51  18   7   0   0   0   0   0   0]
 [  1   0   2   2  11  63 121  92  19  10   1   0   0   0   0]
 [  0   0   0   0   0  10  51  85  59  29   4   1   1   0   0]
 [  0   0   0   0   0   0  12  47  65  44  23   5   2   0   0]
 [  0   0   0   0   0   0   2   5  45  65  38  14   2   3   0]
 [  0   0   0   0   0   0   0   2  11  49  50  27  14   6   1]
 [  0   0   0   0   0   0   0   0   2   5  19  19  17  14   6]
 [  0   0   0   0   0   0   0   0   0   2   4  13  19  24  12]
 [  0   0   0   0   0   0   0   0   0   0   2  10  10  25  18]
 [  0   0   0   0   0   0   0   0   0   0   0   3   4  14  82]]
0.4168
